# Data Load


In [ ]:
import numpy as np
import pandas as pd
import math

from scipy import stats #Analysis 
from scipy.stats import norm 

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

In [ ]:
# 한글 폰트가 깨지면 실행하세요! 
import matplotlib.font_manager as fm
fontlist = fm.findSystemFonts(fontpaths = None, fontext='ttf')
# 아래 주석을 지워서 폰트 리스트를 확인하고 한글 폰트를 font_path에 추가합니다
fontlist[:]

from matplotlib import font_manager, rc
#font_path = 'C:\\Users\\mtang\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumSquare.ttf'
font_path = 'C:\\WINDOWS\\Fonts\\NanumGothicLight.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [ ]:
# pandas option 설정 하는 코드
# monthly_gain의 경우 부동소수점으로 나타나서 보기 어려울땐 윗 줄의 주석을 제거하고 아래에 주석을 추가하고
# 다시 원래대로 돌리고 싶다면 아래에 주석제거, 위 코드에 주석추가
pd.options.display.float_format = '{:.2f}'.format
#pd.reset_option('display.float_format')

In [ ]:
path = "./Dataset/"

data = pd.read_csv(path+"kwproja_data_location.csv")

data

# 2. EDA 

2,927,739 rows X 11 columns

- 매장 속성 정보
  - shop_code -> 식별자 feature => drop   
  - shop_name => DL
  - longtitude : 경도, latitude : 위도 -> 매장 위치 (회사 근처, 학교 근처 등 매출 영향성 있음) -> K-mean clustering => geo, ML, DL
  - address1, address2 : GeoEncoder를 통해 따로 얻은 행정동, 1(30), 2(436) => DL
  - shop_type_big -> 15 category, shop_type_big_label, ML => DL
  - shop_type_small -> 61 category, shop_type_small_label, ML => DL 

- 매출 정보
  - date -> 24 category, 201606~ 201805 까지의 data
  - monthly_gain / avearge_sale_price = 한달 총 판매수

- 매출 통계 정보-> X

In [ ]:
# rename data columns and check the data
data.columns = ['date', 'shop_code', 'shop_name', 'shop_type_big', 'shop_type_small', 
                'longitude', 'latitude', 'monthly_gain', 'average_sale_price', 'address1', 'address2']

print(data.columns, '\n')
print(data.info())

# 3. Preprocessing 
- df_geo : 구(address1)를 기준으로 통계량을 붙여놓은 데이터프레임 
    - monthly_gain + average_sale_price + shop_type_big 개수 
    - count : 데이터 개수 
    - mg : monthly_gain 약자 
    - sp : average_sale_price 약자 
    - mean, st, ~ max : 평균값, 표준편차, ~ 최대값 
    - shop_type_big(15) : 각 업종대분류에 해당하는 데이터가 해당 지역(구)에는 몇 개가 있는가? 
    - 0, 1, 2 : 각 구마다 1등, 2등, 3등으로 많은 업종 대분류는? 
    - **27 rows × 34 columns**
- df_shop : 업종 대분류(shop_type_big)을 기준으로 통계량을 붙여놓은 데이터프레임 
    - **15 rows × 49 columns**
- 구 -> label encoder => (30개)
    - '강남구' '강동구' '강북구' '강서구' '고양시 덕양구' '과천시' '관악구' '광명시' '광진구' '구로구' '구리시'
      '금천구' '노원구' '도봉구' '동대문구' '동작구' '마포구' '서대문구' '서초구' '성동구' '성북구' '송파구' '양천구'
      '영등포구' '용산구' '은평구' '종로구' '중구' '중랑구' '하남시' 

In [ ]:
# 원본 data와 따로 관리 -> original data = data, preprocessed data = processed_data 
processed_data = data.copy()
processed_data

In [ ]:
processed_data['average_sale_price'].describe()

In [ ]:
# box plot 상 outlier와 Quantile 상 75% 이상의 값을 확인합니다
# we detect outlier (from upper, lower) and values greater than 75% in the quantile

Q1 = processed_data['average_sale_price'].quantile(0.25)
Q3 = processed_data['average_sale_price'].quantile(0.75)
IQR = Q3 - Q1
lower_fence = Q1 - (1.5 * IQR)
upper_fence = Q3 + (1.5 * IQR)

upper = processed_data[processed_data['average_sale_price'] >= upper_fence]['average_sale_price']
upper_10 = processed_data[processed_data['average_sale_price'] >= 1000000]['average_sale_price']
lower = processed_data[processed_data['average_sale_price'] <= lower_fence]['average_sale_price']
#upper = processed_data[processed_data['average_sale_price'] >= upper_fence]['average_sale_price']
print(processed_data['average_sale_price'].max())

print(upper_fence)
print(lower_fence)
print(processed_data['average_sale_price'].min())
print()
print()
print(processed_data['average_sale_price'].quantile(0.5))
print(len(upper))
print(len(lower))

#print("%.2f \t" % upper_fence, len(df[df > upper_fence]),
#          "\t %.2f \t" % lower_fence, len(df[df < lower_fence]),
#          "\t %.2f \t" % list_q4, len(df[df > list_q4]))

### geo

In [ ]:
# monthly_gain 
group1 = processed_data[['monthly_gain', 'address1']].groupby('address1')
df1 = group1.describe().droplevel(axis=1, level=0)

# round
df1.iloc[:, 1:] = df1.iloc[:, 1:].applymap(lambda x : round(x, -4))
df1.columns = ['count', 'mean_mg', 'st_mg', 'min_mg', '25%_mg', '50%_mg', '75%_mg', 'max_mg']

# sum - 총 매출
df1['monthly_gain'] = group1.sum().apply(lambda x : round(x, -4))

df1.head()

In [ ]:
# average_sale_price
df2 = processed_data[['average_sale_price', 'address1']].groupby('address1')
df2 = df2.describe().droplevel(axis=1, level=0)

# round
df2.iloc[:, 1:] = df2.iloc[:, 1:].applymap(lambda x : round(x, -4))
df2.columns = ['count', 'mean_sp', 'st_sp', 'min_sp', '25%_sp', '50%_sp', '75%_sp', 'max_sp']
df2.head()

In [ ]:
# shop_type_big
df3 = processed_data.groupby(['address1', 'shop_type_big']).size()
df3 = df3.unstack() # unstack : SQL 집계 결과를 가로, 세로 축으로 보기 좋게 나열 
df3 = df3.fillna(0) # NaN to 0
df3.head()

In [ ]:
# rank for shop_type_big
df4 = df3.apply(lambda s, n: pd.Series(s.nlargest(n).index), axis=1, n=3)
df4.head()

In [ ]:
df_geo = pd.concat([df1, df2.iloc[:, 1:]], axis=1)
df_geo = pd.concat([df_geo, df3], axis=1)
df_geo = pd.concat([df_geo, df4], axis=1)
df_geo.head()

In [ ]:
df_geo.to_csv('df_geo.csv', float_format = '%.2f', encoding = 'utf-8-sig')

### geo - date

In [ ]:
# date 
df5 = processed_data[['date', 'address1', 'monthly_gain']].groupby(['address1', 'date']).mean().round(-4)
df6 = processed_data[['date', 'address1', 'monthly_gain']].groupby(['address1', 'date']).count()

df5 = df5.unstack()
df5 = df5.rename(columns = {'monthly_gain' : 'mean'})
df6 = df6.unstack()
df6 = df6.rename(columns = {'monthly_gain' : 'count'})

df_geo_date = pd.concat([df5, df6], axis=1)
df_geo_date.head()

In [ ]:
df_geo_date.to_csv('df_geo_date.csv', float_format = '%.2f', encoding = 'utf-8-sig')

### shop type big

In [ ]:
# monthly_gain 
group1 = processed_data[['monthly_gain', 'shop_type_big']].groupby('shop_type_big')
df1 = group1.describe().droplevel(axis=1, level=0)

df1.iloc[:, 1:] = df1.iloc[:, 1:].applymap(lambda x : round(x, -4))
df1.columns = ['count', 'mean_mg', 'st_mg', 'min_mg', '25%_mg', '50%_mg', '75%_mg', 'max_mg']

# sum - 총 매출
df1['monthly_gain'] = group1.sum().apply(lambda x : round(x, -4))

df1.head()

In [ ]:
# average_sale_price
df2 = processed_data[['average_sale_price', 'shop_type_big']].groupby('shop_type_big')
df2 = df2.describe().droplevel(axis=1, level=0)

df2.iloc[:, 1:] = df2.iloc[:, 1:].applymap(lambda x : round(x, -4))
df2.columns = ['count', 'mean_sp', 'st_sp', 'min_sp', '25%_sp', '50%_sp', '75%_sp', 'max_sp']
df2.head()

In [ ]:
# address1 - geo
df3 = processed_data.groupby(['shop_type_big', 'address1']).size()
df3 = df3.unstack() # unstack : SQL 집계 결과를 가로, 세로 축으로 보기 좋게 나열 
df3 = df3.fillna(0) # NaN to 0
df3.head()

In [ ]:
# rank for address1(geo)
df4 = df3.apply(lambda s, n: pd.Series(s.nlargest(n).index), axis=1, n=3)
df4.head()

In [ ]:
df_type = pd.concat([df1, df2.iloc[:, 1:]], axis=1)
df_type = pd.concat([df_type, df3], axis=1)
df_type = pd.concat([df_type, df4], axis=1)
df_type.head()

In [ ]:
df_type

### Total

In [ ]:
df_total = processed_data[['monthly_gain', 'average_sale_price']].describe().transpose()

# round - except for min, count 
df_total.iloc[:, 1:3] = df_total.iloc[:, 1:3].applymap(lambda x : round(x,  -4))
df_total.iloc[:, 4:] = df_total.iloc[:, 4:].applymap(lambda x : round(x,  -4))

df_total

In [ ]:
sum_mg = processed_data['monthly_gain'].sum().round(-4)
sum_sp = processed_data['average_sale_price'].sum().round(-4)

df_total["sum"] = [sum_mg, sum_sp]

df_total

In [ ]:
df_total.to_csv('df_total.csv', float_format = '%.2f', encoding = 'utf-8-sig')

### shop_name + shop_type_big + shop_type_small : df_nlp

In [ ]:
# NLP용 preprocessing 
# shop_name, shop_type_big, shop_type_small = concat_text 
df_nlp1 = processed_data.copy()
df_nlp1['concat_text'] = df_nlp1['shop_name'] + ' ' + df_nlp1['shop_type_big'] + ' ' + df_nlp1['shop_type_small'] + ' '
df_nlp1.head()

In [ ]:
df_nlp2 = df_nlp1[['concat_text', 'address1']]
df_nlp2.head()

In [ ]:
from keras.preprocessing.text import Tokenizer

df_nlp = pd.DataFrame()
alist = df_nlp2['address1'].unique()
for add in alist : 
    tk = Tokenizer()
    tk.fit_on_texts(df_nlp2[df_nlp2['address1'] == add]['concat_text'])
    
    wordlist = sorted(tk.word_counts.items(), key=lambda x: x[1], reverse=True)[:30]
    df_word = pd.DataFrame(wordlist)
    df_nlp = pd.concat([df_nlp, df_word], axis=1)
    
    print(add, ": ", list(wordlist))
    print("\nvocab words 개수 : ", len(tk.word_index.items()))

In [ ]:
cols_list = list(df_nlp.columns)
cnt = 0
for i in range (len(cols_list)) :
    if i%2 == 0 : 
        cols_list[i] = alist[cnt]
        cnt = cnt + 1 
    else : 
        cols_list[i] = str(alist[cnt-1]) + " counts"
df_nlp.columns = cols_list
df_nlp.columns

In [ ]:
df_nlp

In [ ]:
df_nlp.to_csv('df_nlp.csv',encoding = 'utf-8-sig')

### shop_name

In [ ]:
# NLP용 preprocessing 
# shop_name, shop_type_big, shop_type_small = concat_text 
df_nlp1 = processed_data.copy()
df_nlp1.head()

In [ ]:
from keras.preprocessing.text import Tokenizer

df_nlp = pd.DataFrame()
alist = df_nlp1['address1'].unique()
for add in alist : 
    tk = Tokenizer()
    tk.fit_on_texts(df_nlp1[df_nlp1['address1'] == add]['shop_name'])
    
    wordlist = sorted(tk.word_counts.items(), key=lambda x: x[1], reverse=True)[:30]
    df_word = pd.DataFrame(wordlist)
    df_nlp = pd.concat([df_nlp, df_word], axis=1)
    
    print(add, ": ", list(wordlist))
    print("\nvocab words 개수 : ", len(tk.word_index.items()))

In [ ]:
cols_list = list(df_nlp.columns)
cnt = 0
for i in range (len(cols_list)) :
    if i%2 == 0 : 
        cols_list[i] = alist[cnt]
        cnt = cnt + 1 
    else : 
        cols_list[i] = str(alist[cnt-1]) + " counts"
df_nlp.columns = cols_list
df_nlp.columns

In [ ]:
df_nlp

In [ ]:
df_nlp.to_csv('df_shopname.csv',encoding = 'utf-8-sig')